In [3]:
import pandas as pd
import json

pd.set_option('display.max_columns', None)

In [4]:
# customer repeat orders -customer_id

with open("iseller_api_response.json", "r") as file:
    data = json.load(file)

In [6]:
def normalize_list_dictionary(dictionary):
    normalized_dict_list = []
    for element in dictionary:
        num_orders = len(element)
        index = 0
        while num_orders > index:
            normalized_dict_list.append(element[index])
            index += 1
    # Remove Keys with None values
    filtered_orders_dict = [order for order in normalized_dict_list if order is not None]
    return filtered_orders_dict

In [7]:
def proccess_iseller_data(data):
    orders_df = pd.DataFrame(data['orders'])
    
    order_details_dict = orders_df["order_details"].to_list()
    normalized_order_details = normalize_list_dictionary(order_details_dict)
    order_details = pd.DataFrame(normalized_order_details)

    #rename columns
    orders_df.rename(columns = {'total_order_amount':' grand_total_amount'}, inplace = True) 

    #merge columns
    relevant_order_details = order_details[["order_id", "product_id", "product_name", "product_type", "fulfillment_status", "quantity", "base_price", "total_order_amount"]]
    merged_orders = orders_df.merge(relevant_order_details, how="left", on="order_id")
    
    # convert to correct dtypes
    merged_orders['order_date'] = pd.to_datetime(merged_orders['order_date'])
    merged_orders['closed_date'] = pd.to_datetime(merged_orders['closed_date'])

    return merged_orders
    
    

In [8]:
iseller_df = proccess_iseller_data(data)

In [12]:
print("First Order: ", iseller_df["order_date"].min())
print("Latest Order: ", iseller_df["order_date"].max())

First Order:  2021-02-25 08:22:02
Latest Order:  2021-02-26 22:06:33


In [15]:
iseller_df.to_csv("iseller_data.csv", index=False)